## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'DistanceFromHome', 'PerformanceRating', 'HourlyRate', 'JobSatisfaction', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'RelationshipSatisfaction']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes



Age                          int64
BusinessTravel              object
DistanceFromHome             int64
PerformanceRating            int64
HourlyRate                   int64
JobSatisfaction              int64
WorkLifeBalance              int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
RelationshipSatisfaction     int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [6]:
# Convert your X data to numeric data types however you see fit
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
# Add new code cells as necessary




In [7]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
    

In [8]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
dept_enc = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
department_enc = dept_enc.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
y_train_dept_enc = department_enc.transform(y_train['Department'].values.reshape(-1, 1))
y_test_dept_enc = department_enc.transform(y_test['Department'].values.reshape(-1, 1))

# to the training and testing data, respectively

c:\Users\Marnie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# Create a OneHotEncoder for the Attrition column
att_enc = OneHotEncoder(sparse=False)


# Fit the encoder to the training data
attrition_enc = att_enc.fit(y_train['Attrition'].values.reshape(-1, 1))


# Create two new variables by applying the encoder
y_train_attr_enc = attrition_enc.transform(y_train['Attrition'].values.reshape(-1, 1))
y_test_attr_enc = attrition_enc.transform(y_test['Attrition'].values.reshape(-1, 1))
# to the training and testing data



c:\Users\Marnie\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_features = len(X_train_scaled[0])

# Create the input layer
inputs = layers.Input(shape=(input_features,))
# Add the first hidden layer
hidden_layer_1 = layers.Dense(units=8, activation='relu')(inputs)
# Add the second hidden layer
hidden_layer_2 = layers.Dense(units=8, activation='relu')(hidden_layer_1)

# Create at least two shared layers
shared_layer_1 = layers.Dense(units=8, activation='relu')(hidden_layer_2)
shared_layer_2 = layers.Dense(units=8, activation='relu')(shared_layer_1)



In [11]:
# Create a branch for Department
department_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
department_output = layers.Dense(units=3, activation='softmax', name='Department')(department_branch)

# with a hidden layer and an output layer
# Create the hidden layer
hidden_layer_3 = layers.Dense(units=8, activation='relu')(shared_layer_2)

# Create the output layer
output = layers.Dense(units=2, activation='softmax', name='Department')(hidden_layer_3)

# Create a branch for Attrition
department_branch_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
department_output = layers.Dense(units=2, activation='softmax', name='Department')(department_branch)



In [12]:
# Create a branch for Attrition
attrition_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
attrition_output = layers.Dense(units=2, activation='softmax', name='Attrition')(attrition_branch)

# with a hidden layer and an output layer
# Create the hidden layer
hidden_layer_3 = layers.Dense(units=8, activation='relu')(shared_layer_2)

# Create the output layer
output = layers.Dense(units=2, activation='softmax', name='Attrition')(hidden_layer_3)

# Create a model with the layers



In [13]:
# Define the output layers
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(inputs)
department_output = layers.Dense(y_train_dept_enc.shape[1], activation='sigmoid', name='department_output')(department_hidden)

attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(inputs)   
attrition_output = layers.Dense(y_train_attr_enc.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)

# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss = {'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics= {
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              
              })

# Fit the model to the training data



# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │        416 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │        416 │ input_layer[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 997 (3.89 KB)

 Trainable params: 997 (3.89 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_dept_enc, 'attrition_output': y_train_attr_enc},
          epochs=100, shuffle=True)


Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - attrition_output_accuracy: 0.6261 - department_output_accuracy: 0.5847 - loss: 1.5785 
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8187 - department_output_accuracy: 0.6378 - loss: 1.3798 
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8308 - department_output_accuracy: 0.6609 - loss: 1.2919 
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - attrition_output_accuracy: 0.8039 - department_output_accuracy: 0.6500 - loss: 1.2814
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 903us/step - attrition_output_accuracy: 0.8050 - department_output_accuracy: 0.6479 - loss: 1.2684
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - attrition_output_accuracy: 0.8346 - department_output_accuracy: 0.6638 - loss: 1.1891
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 845us/step - attrition_output_accuracy: 0.8418 - department_output_accuracy: 0.6809 - loss: 1.1423
Epoch 8/100
35/3

In [15]:
# Evaluate the model with the testing data
test_results= model.evaluate(X_test_scaled, {'department_output': y_test_dept_enc, 'attrition_output': y_test_attr_enc})
test_results

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8774 - department_output_accuracy: 0.5787 - loss: 1.2752  


[1.2401841878890991, 0.8777173757553101, 0.58423912525177]

In [16]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]}")  
print(f"Attrition Accuracy: {test_results[1]}")


Department Accuracy: 0.58423912525177
Attrition Accuracy: 0.8777173757553101


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. The data seems to be inmablanced which means the F one score or balanced accuracy would be better.
2. I have tried relu, sigmoid and softmax. Relu worked for the hidden layers, I used softmax and sigmoid for the output layers.
3. Including more columns, adding more layers, finding more training data or experimenting with activation functions could improve tis model.